<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0714japanese_gpt_1b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-3 を使って，小野オノマトペ辞典4500 の説明文 (リード文) から文生成させてみた
- date: 2022_0714
- author: 浅川 伸一
- ref: https://huggingface.co/rinna/japanese-gpt-1b

In [ ]:
import torch
%config InlineBackend.figure_format = 'retina'
try:
    import bit
except ImportError:
    !pip install ipynbname > /dev/null 2>&1
    !git clone https://github.com/ShinAsakawa/bit.git
import bit
isColab = bit.isColab
HOME = bit.HOME

if isColab:
    !pip install --upgrade xlrd
    !pip install --upgrade 'fugashi[ipadic]'
    !pip install --upgrade 'fugashi[unidic]'
    !python -m unidic download
    !pip install --upgrade ipadic
    !pip install sentencepiece
    !pip install --upgrade transformers
    !pip install --upgrade termcolor
    !pip install --upgrade jaconv    

In [ ]:
import torch
from transformers import T5Tokenizer, AutoModelForCausalLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt-1b")
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt-1b")

if torch.cuda.is_available():
    model = model.to("cuda")

text = "西田幾多郎は、"
token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_length=100,
        min_length=100,
        do_sample=True,
        top_k=500,
        top_p=0.95,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bad_word_ids=[[tokenizer.unk_token_id]]
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)  
# sample output: 西田幾多郎は、その主著の「善の研究」などで、人間の内面に自然とその根源があると指摘し、その根源的な性格は、この西田哲学を象徴しているとして、カントの「純粋理性批判」と「判断力批判」を対比して捉えます。それは、「人が理性的存在であるかぎりにおいて、人はその当人に固有な道徳的に自覚された善悪の基準を持っている」とするもので、この理性的な善悪の観念を否定するのがカントの


In [ ]:
import os 
# 2021/Jan 近藤先生からいただいたオノマトペ辞典のデータの読み込み
#'日本語オノマトペ辞典4500より.xls' は著作権の問題があり，公にできません。
# そのため Google Colab での解法，ローカルファイルよりアップロードしてください
if isColab:
    from google.colab import files
    
    # ここで `日本語オノマトペ辞典4500より.xls` を指定してアップロードする
    uploaded = files.upload()  
    data_dir = '.'
else:
    data_dir = os.path.join(HOME, 'study/2021ccap/notebooks')

import pandas as pd
import jaconv

onomatopea_excel = '2021-0325日本語オノマトペ辞典4500より.xls'
onmtp2761 = pd.read_excel(os.path.join(data_dir, onomatopea_excel), sheet_name='2761語')

#すべてカタカナ表記にしてデータとして利用する場合
#`日本語オノマトペ辞典4500` はすべてひらがな表記だが，一般にオノマトペはカタカナ表記されることが多いはず
#onomatopea = list(sorted(set([jaconv.hira2kata(o) for o in onmtp2761['オノマトペ']])))

# Mac と Windows の表記の相違を吸収
onomatopea = list(sorted(set([jaconv.normalize(o) for o in onmtp2761['オノマトペ']])))
print(f'データファイル名: {os.path.join(data_dir, onomatopea_excel)}\n',
      f'オノマトペ単語総数: len(onomatopea):{len(onomatopea)}')

In [ ]:
#print(onomatopea);

# 2. `onomatope11 (1).xlsx` を用いたオノマトペ説明文の BERT 化

In [ ]:
if isColab:
    # '2022_0531onomatope11.xlsx' または onomatope11(1).xlsx をアップロードする
    from google.colab import files
    uploaded = files.upload()

In [7]:
from termcolor import colored
import pandas
# fname = 'onomatope11 (1).xlsx'
# 上の若干の入力ミスと思われる部分を修正した。オリジナルは onomatope11(1).xlsx
fname = '2022_0531onomatope11.xlsx'

if isColab:
    df = pandas.read_excel(fname, sheet_name='onomatope11')
else:
    df = pandas.read_excel(os.path.join(HOME, 'study/2022kondo_project', fname), sheet_name='onomatope11')

X1 = df[['bunn', 'onomatope']].values.tolist()
onomatopea_sentences = [jaconv.normalize(str(line[0]))+'、'+jaconv.normalize(str(line[1])) for line in X1]

In [8]:
#X1[0][0]

In [10]:
#text = X1[0][0]
#token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_length=100,
        min_length=100,
        do_sample=True,
        top_k=500,
        top_p=0.95,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bad_word_ids=[[tokenizer.unk_token_id]]
    )

#output = tokenizer.decode(output_ids.tolist()[0])
#print(output)  

In [11]:
def gpt3_gen(text:str):
    token_ids = tokenizer.encode(text, 
                                 add_special_tokens=False, 
                                 return_tensors="pt")
    with torch.no_grad():
        output_ids = model.generate(
            token_ids.to(model.device),
            max_length=100,
            min_length=100,
            do_sample=True,
            top_k=500,
            top_p=0.95,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            bad_word_ids=[[tokenizer.unk_token_id]]
        )
    output = tokenizer.decode(output_ids.tolist()[0])
    return output

In [12]:
#print(text, end=":")
#print(gpt3_gen(text))

In [13]:
leads = [_lead[0] for _lead in X1]
for _lead in leads:
    print(colored(_lead, 'blue', attrs=['bold']))
    print(gpt3_gen(_lead))

日が射したり光がともって
日が射したり光がともって明るい場所でも夜空の星が良く見えるといいますが、日中の太陽の位置から見るとかなり上のほうにあることがわかると思います。この太陽が射す高さのことを「地平高度」といいます。これは視力検査で測定して知ります。しかし、地平高度は地平線という空の最も低い位置での高さを言いますので、それより上は地球の内部で、高度はとんでもなく上がっています。その高度は地球の内部でも、地表に近くなる
あたたかい日ざしが差しこみ
あたたかい日ざしが差しこみ、穏やかな気持ちになる。木の下でしばらく居座ってしまいたい気持ちを抑え、走り出す。時計はもう、11時を回っている。駅までの道のりも長い。大泉緑地の散策は、約1時間のウォーキング。ここに来るといつも、時計の針が進むのが遅く感じる。なんだか、のんびり。公園を抜けて線路沿いの遊歩道を歩く。沿道には植えられた植物が季節にそって手入れされ、花々は香りを放ち、緑がキラキラ
水や日ざしが満ち満ちて
水や日ざしが満ち満ちて、美しい緑の茂みの中を歩きます。晴れた日には、美しい景色や、新緑がお気に入りの一枚に撮れます。また、秋の紅葉シーズンは、紅葉の中をお散歩します。草木の隙間から覗く木漏れ日の中の木道はとてもすがすがしいものです。真夏の暑い時期にだって、涼しげな空気を感じてリフレッシュ出来ます。水辺なら夏場でも涼しく、お散歩には最高です。緑豊かな公園があれば、緑
すっかり日が暮れて
すっかり日が暮れてしまいましたが、みんながんばりました!全行程5時間。大変長い1日でありました。でも楽しかったなぁ。これで、いよいよ3月も終わりです。もう来月は新年度!何かまた一つ大きな変化があるのではないかとドキドキしています。まぁ、気楽に楽しみにしていくことにしましょう。みなさま、年度末でお忙しいでしょうが、みなさまご自愛くださいね。私も早く帰って一杯飲みましょう。それでは、よい夢を!最後に、まだご覧でない方のために、「春の千と千尋の神隠し
日光の照り輝いて
日光の照り輝いて美しさに目を奪われる。[UNK]曲(えんきょく)の調子をとること。[UNK]「秋来ぬと目にはさやかに見えねども風の音にぞおどろかれぬる」。〔秋の風〕朝晩吹く風の音が、人の耳や心を奪う。(玉葉)。雅風。雅く美しいこと。美風。〔高麗〕天皇か、あるいは親ら使用したとき

# 続いて，リード文＋オノマトペ，から文生成

In [ ]:
leads = [_lead[0]+'、'+_lead[1] for _lead in X1]
for _lead in leads:
    print(colored(_lead, 'blue', attrs=['bold']))
    print(gpt3_gen(_lead))

日が射したり光がともって、ぽっ
日が射したり光がともって、ぽっ豊川市と素敵な女性に見える......同じ悲しい憧れは危険でしょう。そんなため、簡単に恋活・婚活したい方はマッチドットコムを使ってみてください。例えば、女性が「女性」を感じる、人妻男性が自分をもって「女性」を感じる瞬間とは同じ時なのでしょうか。ですが、出会いが高いと物欲が満たされなくなり、太りやすくなる子供があることを忘れてはいけません。・「私がぽっ無料い体質なので、いまいちに複数にしてもらいたい。女性を重ねた
あたたかい日ざしが差しこみ、ぽかぽか
あたたかい日ざしが差しこみ、ぽかぽか、心地いい時間が流れています♪きょうも「ひとりではできなくて・・・」と、お友だちがきてくれました(^^)明日も、お待ちしています。 今日は、ちいさなてぶくろの、ゆびぬきが出来上がりました。編み物はこつをつかめば、だれでも簡単に編むことができます。本物をみて触って、ぜひ、編み物をはじめてみませんか(^o^) 「ひとりでは無理そうだから・・・」と、遠慮しがちなお友だち
水や日ざしが満ち満ちて、なんなん
水や日ざしが満ち満ちて、なんなんこの天気!って感じです。今日の最高気温は34°Cでした。確かに、暑いのですが、なんだかもの悲しいような暑さを感じます。熱さだけが暑いような変な感じです。昼休み、金木犀が咲いていて、その香りが秋の到来を感じさせます。今日も、涼しげに色づいていました。そういえば、最近、暑いので、浴衣で歩く女性が以前より増えました。暑いのに、浴衣ってどうよ?って初めは思いながら
すっかり日が暮れて、とっぷり
すっかり日が暮れて、とっぷり暮れた9時過ぎに解散。しかし、自分の会社へ戻らなければならないので、あまりゆっくりもできず、またしてもタクシーにのってうちに到着したときには、すでに10時をまわっていた。今年は、9月に起きた水害の影響で、3日ほど大雨が降っており、その影響が残っている中での登山となったが、晴天のもとでの富士山もすばらしく、達成感にひたりながら、楽しい一日を過ごすことができた。来シーズンも楽しい山行を楽しみたいものである。来月には、久しぶりに立山
日光の照り輝いて、てかてか
日光の照り輝いて、てかてかしていて、まるで赤ちゃん肌。お肌がとても綺麗で、ピンクと白の大胆に境界を割り、どちらともとれそうな、これまた美